# Test AutoFileGroup
This notebook tests the grouping functionality in AutoFileGroup class

In [2]:
import os
import sys
import yaml
from loguru import logger

from autocoder.agent.auto_filegroup import AutoFileGroup
import byzerllm

## Create test data directory and files

In [3]:
# Create test directory
test_dir = ".tmp/test_auto_filegroup"
os.makedirs(test_dir, exist_ok=True)

# Create test YAML files
test_data = [
    {
        "name": "001_feature1.yml",
        "content": {
            "query": "Add input validation to login function",
            "urls": [
                "/src/auth/login.py",
                "/src/utils/validation.py"
            ]
        }
    },
    {
        "name": "002_feature2.yml",
        "content": {
            "query": "Implement password reset functionality",
            "urls": [
                "/src/auth/password_reset.py",
                "/src/auth/email.py"
            ]
        }
    },
    {
        "name": "003_feature3.yml", 
        "content": {
            "query": "Add caching to database queries",
            "urls": [
                "/src/db/cache.py",
                "/src/db/queries.py"
            ]
        }
    },
    {
        "name": "004_feature4.yml",
        "content": {
            "query": "Implement rate limiting for API endpoints",
            "urls": [
                "/src/api/rate_limiter.py",
                "/src/api/middleware.py"
            ]
        }
    },
    {
        "name": "005_feature5.yml",
        "content": {
            "query": "Add email verification after signup",
            "urls": [
                "/src/auth/email.py",
                "/src/auth/signup.py"
            ]
        }
    }
]

# Write test files
for item in test_data:
    file_path = os.path.join(test_dir, item["name"])
    with open(file_path, "w", encoding="utf-8") as f:
        yaml.dump(item["content"], f)

print(f"Created {len(test_data)} test files in {test_dir}")

Created 5 test files in .tmp/test_auto_filegroup


## Initialize LLM and test grouping

In [5]:
llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")

# Create AutoFileGroup instance
grouper = AutoFileGroup(llm=llm, actions_dir=test_dir)

# Test file grouping
groups = grouper.group_files()

print("\nFile Grouping Results:")
print("=====================")
for idx, group in enumerate(groups, 1):
    print(f"\nGroup {idx}: {group.name}")
    print("Queries:")
    for query in group.queries:
        print(f"- {query}")
    print("Files:")
    for url in group.urls:
        print(f"- {url}")

2024-12-21 11:37:03.299 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-12-21 11:37:03,369	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-12-21 11:37:03,370	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.



File Grouping Results:

Group 1: Authentication and Authorization
Queries:
- Add input validation to login function
- Implement password reset functionality
- Add email verification after signup
Files:
- /src/auth/login.py
- /src/utils/validation.py
- /src/auth/password_reset.py
- /src/auth/email.py
- /src/auth/signup.py

Group 2: Database and Caching
Queries:
- Add caching to database queries
Files:
- /src/db/cache.py
- /src/db/queries.py

Group 3: API Management
Queries:
- Implement rate limiting for API endpoints
Files:
- /src/api/rate_limiter.py
- /src/api/middleware.py


## Cleanup

In [6]:
# Remove test directory and files
import shutil
shutil.rmtree(test_dir)
print(f"\nCleaned up test directory: {test_dir}")


Cleaned up test directory: .tmp/test_auto_filegroup
